In [1]:
import sys
sys.path.append('/home/ian/code/github')

## 获取症状列表

In [2]:
%%time
from model import model_wrapper

CPU times: user 609 ms, sys: 99.6 ms, total: 708 ms
Wall time: 717 ms


In [3]:
%%time
model = model_wrapper()

CPU times: user 43 s, sys: 2.43 s, total: 45.4 s
Wall time: 45.5 s


In [4]:
model.get_zzs('我牙肿了','男','1')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.580 seconds.


牙肿


Prefix dict has been built succesfully.
/home/ian/code/github/crawl/requestProj/39jiankang/model.py:62: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['口']
288


['牙疼', '牙周脓肿', '齿龈炎']

In [5]:
model.get_zzs('我牙出血了')

牙出血


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:62: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['口']
279


['刷牙出血', '牙龈出血', '牙结石', '牙齿臭', '牙移位', '牙脱位', '齿龈出血', '口唇出血', '牙齿黑斑', '牙龈痒胀']

In [6]:
model.get_zzs('我有点发烧')

发烧


'发烧'

In [7]:
model.get_zzs('我睡不着觉')

失眠


'失眠'

In [8]:
model.get_zzs('我感觉头有点晕')

头晕


'头晕'

In [9]:
model.get_zzs('我感觉胳膊有点疼痛')

上臂上肢前臂疼痛


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:62: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['臂']
18


['前臂外伤后疼痛', '前臂和腕部的疼痛', '伸臂握手疼痛']

In [10]:
model.get_zzs('我感觉肚子有点痛')

肚子腹部痛


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:62: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['腹部']
191


['肚子疼', '慢性腹痛', '右下腹痛', '小腹冷痛', '右上腹压痛', '左下腹压痛', '上腹部压痛', '右下腹压痛', '左下腹痛']

In [11]:
model.get_zzs('我感觉脚有点痛')

脚足痛


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:62: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['脚']
85


['足痛', '脚趾痛']

In [12]:
model.get_zzs('我感觉小腿有点疼','女','5','孕妇')

小腿下肢疼


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:62: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['腿']
94


['小腿溃疡', '小腿肚抽筋', '小腿酸痛', '小腿水肿', '小腿发胀', '大腿外侧痛', '孕妇小腿痒']

In [13]:
model.get_zzs('我感觉小腿有点疼','女','5')

小腿下肢疼


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:62: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['腿']
90


['小腿溃疡', '小腿肚抽筋', '小腿酸痛', '小腿水肿', '小腿发胀', '大腿外侧痛']

In [14]:
model.get_zzs('我感觉小腿有点疼','男','1')

小腿下肢疼


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:62: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['腿']
91


['小腿溃疡', '小腿肚抽筋', '小腿酸痛', '小腿水肿', '小腿发胀', '大腿外侧痛']

In [15]:
model.get_zzs('我感觉有点恶心')

恶心


'恶心'

In [16]:
model.get_zzs('我感觉有点想吐','女','1')

恶心与呕吐


'恶心与呕吐'

In [7]:
model.get_zzs('我感觉有点想吐','男','1')

想吐


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:60: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


[]


array(['多梦', '夜惊', '舌痛', '眼袋', '不举', '早饱', '口咸', '呛咳', '夜盲', '嗳气', '吐弄舌',
       '吐泡沫痰', '口燥咽干', '泛恶欲吐', '吐白沫痰', '口干', '吞咽痛', '夜啼饶哭', '梗噎', '频繁吐黏液'],
      dtype=object)

In [16]:
model.get_zzs('我脸上长了个痘痘')

脸上长个痘痘


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:60: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['面部']
135


['脸上长斑', '头上长疙瘩', '脸上黑斑', '下巴长痘', '宝宝脸上长红疙瘩', '脸颊发灰', '雀斑']

In [15]:
model.zz2zz(['头胀'])

AttributeError: 'model_wrapper' object has no attribute '_tongxianmatr'

In [19]:
model.get_zzs('我眼睛困')

眼睛困


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:60: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['眼']
283


['眼睛痒',
 '眼痛',
 '眼干',
 '眼屎多',
 '眼缺陷',
 '眼睛发红',
 '眼睛刺痛',
 '眼皮跳',
 '眼充血',
 '眼袋',
 '眼睛胀痛',
 '眼睛干涩',
 '孕期眼睛干燥',
 '眼肌麻痹',
 '两眼干涩',
 '眼疲劳']

In [ ]:
# from utils.pymongo_wrapper import pymongo_wrapper as pmw

# pmw = pmw()

# zztable = pmw.get_collection('jiankang39', 'zznew')

# zzlist = [i['症状名称'] for i in pmw.find_all(zztable, fieldlist=['症状名称'])]



# from utils.pickle_wrapper import pickle_wrapper as picklew

# picklew.dump2file(zzlist,'症状列表.pkl')
from utils.pickle_wrapper import pickle_wrapper as picklew
zzlist = picklew.loadfromfile('症状列表.pkl')
len(zzlist),zzlist[:5]

### 通过编辑距离计算相似度

In [ ]:
from utils.nlp.nlp_wrapper import nlp_wrapper as nlpw
from utils.pandas_wrapper import pandas_wrapper as pdw
from utils.pandas.dataframe_wrapper import dataframe_wrapper as dfw
def get_zzs_by_editdis(s1,zzlist, n=10):
    e = list(map(nlpw.cal_editdistance, [s1]*len(zzlist), zzlist))
    df = pdw.build_df_from_dict({'zz':zzlist,'e':e})
    return list(dfw.sort_by_column(df,'e').loc[:,'zz'][:n].values)

In [ ]:
get_zzs_by_editdis('睡不着觉',zzlist)

In [ ]:
get_zzs_by_editdis('我经常失眠',zzlist)

In [ ]:
get_zzs_by_editdis('肘关节疼痛',zzlist)

### 通过word2vec模糊推荐症状

In [ ]:
import gensim
import jieba
import numpy as np
from scipy.linalg import norm
 
model_file = '/home/ian/code/github/data/word2vec/news_12g_baidubaike_20g_novel_90g_embedding_64.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_file, binary=True)


In [ ]:
def vector_similarity(s1, s2):
    def sentence_vector(s):
        words = jieba.lcut(s)
        v = np.zeros(64)
        for word in words:
            if word in model:
                v += model[word]
        v /= len(words)
        return v
    try:
        v1, v2 = sentence_vector(s1), sentence_vector(s2)
        return np.dot(v1, v2) / (norm(v1) * norm(v2))
    except:
        return 0

In [ ]:
def get_zzs_by_word2vec(s1,zzlist,n=10):
    e = list(map(vector_similarity, [s1]*len(zzlist), zzlist))
    df = pdw.build_df_from_dict({'zz':zzlist,'e':e})
    return list(dfw.sort_by_column(df,'e',ascending=False).loc[:,'zz'][:n].values)

In [ ]:
get_zzs_by_word2vec('睡不着觉',zzlist)

In [ ]:
get_zzs_by_word2vec('我经常失眠',zzlist)

### 得到模糊症状匹配
如何提高准确率？  
* 在模型使用过程中，排除影响模型的个别word  
* zzlist  
保证zzlist都是可问的，就是说根据用户的特征（孕妇，老人，男女）有针对的zzlist

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/home/ian/code/data/daozhen/部位同义词.csv')
df.head()

In [ ]:
df['部位长度'] = df.apply(lambda x: len(x[2]),1)

In [ ]:
df.head()


In [ ]:
df1 = df.sort_values('部位长度',ascending=False)
df1.head()

In [ ]:
from utils.pickle_wrapper import pickle_wrapper as pkw

In [ ]:
# buweilist = list(df1.loc[:,'同义部位'])
# pkw.dump2file(buweilist,'二级部位列表.pkl')

buweilist = pkw.loadfromfile('二级部位列表.pkl')
len(buweilist)

In [ ]:
# buweiseries = pd.Series(df1.loc[:,'C2'].values,index=df1.loc[:,'同义部位'])
# pkw.dump2file(buweiseries,'二级部位series.pkl')
buweiseries = pkw.loadfromfile('二级部位series.pkl')

buweiseries[:5]

In [ ]:
import re

In [ ]:
pattern = re.compile(r'|'.join(buweilist))

In [ ]:
re.findall(pattern,'我腹部疼')

In [ ]:
re.findall(pattern,'我肚子和大腿疼')

In [ ]:
list1 = re.findall(pattern,'我肚子肚和大腿疼')

In [ ]:
{buweiseries[i] for i in list1}

In [ ]:
def get_buwei(s):
    '''
    从一个描述中得到部位
    '''
    list1 = re.findall(pattern,s)
    return list({buweiseries[i] for i in list1})

In [ ]:
get_buwei('我肚子和大腿疼')

In [ ]:
import re
def get_zzs(s1,zzlist,gender,age):
    '''
    @gender: 男，女，孕妇，产褥期
    '''
    s1 = re.sub(r'[我你他很的,，。？！了么呢吗哦哈]|(感觉)|(非常)|(有点)','',s1)
    s1 = re.sub(r'胳膊','上臂上肢前臂',s1)
    s1 = re.sub(r'小腿','小腿下肢',s1)
    s1 = re.sub(r'大腿','大腿下肢',s1)
#     s1 = re.sub(r'脸','脸面部',s1)
    s1 = re.sub(r'喉咙','喉咙咽喉',s1)
    s1 = re.sub(r'嘴','嘴口',s1)
#     s1 = re.sub(r'牙','牙牙齿',s1)
    s1 = re.sub(r'屁股','屁股臀部',s1)
    s1 = re.sub(r'肚子','肚子腹部',s1)
    s1 = re.sub(r'脚','脚足',s1)
#     s1 = re.sub(r'[疼痛]','疼痛',s1)
    s1 = re.sub(r'睡不着觉|睡不着','失眠',s1)
    s1 = re.sub(r'鼻子流血','流鼻血',s1)
    s1 = re.sub(r'黑眼圈重','黑眼圈',s1)
    print(s1)
    if s1 in zzlist:
        return s1
    ll = pd.Series(get_zzs_by_editdis(s1,zzlist)+get_zzs_by_word2vec(s1,zzlist)).unique()
    bwlist = get_buwei(s1)
    print(bwlist)
    if bwlist:
        zzdicts = pmw.find_all(zztable,{'$or':[{'c2':i} for i in bwlist]},fieldlist=['症状名称'])
        zz1 = [i['症状名称'] for i in zzdicts]        
        print(len(zz1))
        return [i for i in ll if i in zz1]
    return ll

In [ ]:
get_zzs('肘关节疼痛',zzlist)

In [ ]:
get_zzs('我睡不着觉',zzlist)

In [ ]:
get_zzs('我喉咙感觉有点痒',zzlist)

In [ ]:
get_zzs('我脸有点肿',zzlist)

In [ ]:
get_zzs('我的脖子非常痛',zzlist)

In [ ]:
get_zzs('我牙很肿',zzlist)

In [ ]:
get_zzs('我有点发烧',zzlist)

In [ ]:
get_zzs('我感觉头有点晕',zzlist)

In [ ]:
get_zzs('我感觉胳膊有点疼痛',zzlist)

In [ ]:
get_zzs('我感觉小腿有点疼',zzlist)

In [ ]:
get_zzs('我感觉大腿有点疼',zzlist)

In [ ]:
get_zzs('我感觉喉咙有点疼',zzlist)

In [ ]:
get_zzs('我感觉脚有点痛',zzlist)

In [ ]:
get_zzs('我感觉肚子有点痛',zzlist)

In [ ]:
get_zzs('我流鼻血了',zzlist)

In [ ]:
get_zzs('我鼻子流血了',zzlist)

In [ ]:
get_zzs('我黑眼圈很重',zzlist)

In [ ]:
e = list(map(vector_similarity, ['我牙疼']*len(zzlist), zzlist))
e[:5]

In [ ]:
df = pdw.build_df_with_dict({'zz':zzlist,'e':e})
list(pdw.sort_by_column(df,'e',ascending=False).loc[:,'zz'][:5].values)

In [ ]:
pdw.sort_by_column(df,'e')